In [1]:
%load_ext cudf.pandas
import pandas as pd

print(pd)

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>


In [2]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import glob
from sklearn.model_selection import train_test_split

In [3]:
import torch
torch.__version__

'2.1.0.post300'

In [4]:
articles = pd.read_csv("data/articles.csv")
customers = pd.read_csv("data/customers.csv")
transactions = pd.read_csv("data/transactions_train.csv")

In [5]:
print("articles shape: ", articles.shape)
print("customers shape: ", customers.shape)
print("transactions shape: ", transactions.shape)

articles shape:  (105542, 25)
customers shape:  (1371980, 7)
transactions shape:  (31788324, 5)


In [6]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [7]:
transactions['customer_id'] = transactions['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
transactions['article_id'] = transactions.article_id.astype('int32')
#train['article_id'] = '0' + train.article_id.astype('str')
transactions.t_dat = pd.to_datetime(transactions.t_dat)
transactions = transactions[['t_dat','customer_id','article_id']]
transactions['year'] = (transactions.t_dat.dt.year-2000).astype('int8')
transactions['month'] = (transactions.t_dat.dt.month).astype('int8')
transactions['day'] = (transactions.t_dat.dt.day).astype('int8')
del transactions['t_dat']
# transactions.to_parquet('transactions.pqt',index=False)
print( transactions.shape )
transactions.head()

(31788324, 5)


,customer_id,article_id,year,month,day
0,-6846340800584936,663713001,18,9,20
1,-6846340800584936,541518023,18,9,20
2,-8334631767138808638,505221004,18,9,20
3,-8334631767138808638,685687003,18,9,20
4,-8334631767138808638,685687004,18,9,20


In [8]:
customers['customer_id'] = customers['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
articles['article_id'] = articles['article_id'].astype('int32')

In [9]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,6883939031699146327,NaN,NaN,ACTIVE,NONE,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,-7200416642310594310,NaN,NaN,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,-6846340800584936,NaN,NaN,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,-94071612138601410,NaN,NaN,ACTIVE,NONE,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,-283965518499174310,1.0,1.0,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [10]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [11]:
transactions.article_id.nunique()

104547

In [12]:
transactions.customer_id.nunique()

1362281

In [13]:
len(customers)

1371980

In [14]:
len(customers)-transactions.customer_id.nunique()

9699

In [15]:
customers.customer_id.nunique()

1371980

In [16]:
transactions = transactions[transactions['year']>=20]
transactions.shape

(10980132, 5)

In [17]:
customers = customers[customers['customer_id'].isin(set(transactions.customer_id.unique()))]
print(len(customers))

862724


In [18]:
articles = articles[articles['article_id'].isin(set(transactions.article_id.unique()))]
print(len(articles))

61248


In [19]:
articles.reset_index(inplace=True)
customers.reset_index(inplace=True)

In [20]:
all_image_paths = glob.glob(f"data/images/*/*")
print(len(all_image_paths))
all_image_ids = set()
for path in tqdm(all_image_paths):
    article_id = path.split('/')[-1].split('.')[0]
    all_image_ids.add(int(article_id))

105100


  0%|          | 0/105100 [00:00<?, ?it/s]

In [21]:
images_path = f'data/images/0'

articles["path"] = images_path + articles["product_code"].astype(str) + "/0" + articles["article_id"].astype(str) + ".jpg"

# Adjust the incorrect paths and set them to None
for k, article_id in tqdm(enumerate(articles["article_id"])):
    if article_id not in all_image_ids:
        articles.loc[k, "path"] = None

0it [00:00, ?it/s]

In [22]:
articles.dropna(subset=['path'],inplace=True)

In [23]:
articles.shape

(61110, 27)

In [24]:
transactions

,customer_id,article_id,year,month,day
20808192,5571569540382626050,835247001,20,1,1
20808193,3910590684697740887,802930002,20,1,1
20808194,3910590684697740887,760084008,20,1,1
20808195,-3097924680776566591,804662002,20,1,1
20808196,-3097924680776566591,801554002,20,1,1
...,...,...,...,...,...
31788319,4685485978980270934,929511001,20,9,22
31788320,4685485978980270934,891322004,20,9,22
31788321,3959348689921271969,918325001,20,9,22
31788322,-8639340045377511665,833459002,20,9,22


In [25]:
core_10_customers = transactions.groupby(['customer_id'])['article_id'].count()>=10
core_10_customers

customer_id
-9223352921020755230    False
-9223343869995384291     True
-9223295149301589789    False
-9223293121067732640     True
-9223290575350349271     True
                        ...  
 9223108375636425507    False
 9223141695752178477    False
 9223144259947635662    False
 9223148401910457466     True
 9223290368606939827    False
Name: article_id, Length: 862724, dtype: bool

In [26]:
core_10_customers[core_10_customers].index

Int64Index([-9223343869995384291, -9223293121067732640, -9223290575350349271,
            -9223273623650064010, -9223251502792607675, -9223238917814440456,
            -9223194771605040811, -9223129394749283666, -9223100958908512198,
            -9223100934418577292,
            ...
             9222546307854699030,  9222665711703637549,  9222733909628763172,
             9222844312705255886,  9222940818992675193,  9222994810437999589,
             9223006465402827359,  9223050760068551513,  9223099843213569889,
             9223148401910457466],
           dtype='int64', name='customer_id', length=343170)

In [27]:
transactions = transactions[transactions['customer_id'].isin(set(core_10_customers[core_10_customers].index))]

In [28]:
core_10_articles = transactions.groupby(['article_id'])['customer_id'].count()>=10
core_10_articles

article_id
108775015     True
108775044     True
110065001     True
110065002     True
110065011     True
             ...  
952267001     True
952938001    False
953450001     True
953763001     True
956217002     True
Name: customer_id, Length: 59740, dtype: bool

In [29]:
core_10_articles[core_10_articles].index

Int64Index([108775015, 108775044, 110065001, 110065002, 110065011, 111565001,
            111586001, 111593001, 111609001, 120129001,
            ...
            947509001, 947599001, 947934001, 949198001, 949551001, 949551002,
            952267001, 953450001, 953763001, 956217002],
           dtype='int64', name='article_id', length=36856)

In [30]:
transactions = transactions[transactions['article_id'].isin(set(core_10_articles[core_10_articles].index))]

In [31]:
merged1 = pd.merge(transactions, articles, on='article_id', how='inner')
merged2= pd.merge(transactions, customers, on='customer_id', how='inner')
common_articles = set(merged1['article_id'])
common_customers = set(merged2['customer_id'])
transactions = transactions[transactions['article_id'].isin(common_articles) & transactions['customer_id'].isin(common_customers)]
common_articles = set(transactions['article_id'])
common_customers = set(transactions['customer_id'])
articles = articles[articles['article_id'].isin(common_articles)]
customers = customers[customers['customer_id'].isin(common_customers)]

In [32]:
print(transactions.article_id.nunique())
print(articles.article_id.nunique())
print(transactions.customer_id.nunique())
print(customers.customer_id.nunique())

36806
36806
343166
343166


In [33]:
transactions.shape

(8746715, 5)

In [34]:
customer_map = {u: i for i, u in enumerate(set(transactions['customer_id']))}
article_map = {a: i for i, a in enumerate(set(transactions['article_id']))}
transactions['customer_id'] = transactions['customer_id'].map(customer_map)
transactions['article_id'] = transactions['article_id'].map(article_map)
customers['customer_id'] = customers['customer_id'].map(customer_map)
articles['article_id'] = articles['article_id'].map(article_map)


In [35]:
transactions.shape

(8746715, 5)

In [36]:
transactions.customer_id.nunique()

343166

In [37]:
customers.shape

(343166, 8)

In [38]:
articles.shape

(36806, 27)

In [39]:
transactions.reset_index(inplace=True, drop=True)
customers.reset_index(inplace=True)
articles.reset_index(inplace=True)

In [40]:
transactions.head()

,customer_id,article_id,year,month,day
0,263072,32057,20,1,1
1,263072,36091,20,1,1
2,343144,31808,20,1,1
3,343144,17165,20,1,1
4,343144,13396,20,1,1


In [45]:
train, test = train_test_split(transactions, test_size=0.2, random_state=42, stratify=transactions['article_id'])

In [47]:
train.shape, test.shape

((6997372, 5), (1749343, 5))

In [43]:
def transaction_only(df):
    df = df.sort_values(by=['customer_id','article_id'])
    df['article_id'] = df['article_id'].map(str)
    df = df.groupby(['customer_id']).agg({'article_id':' '.join})
    return df
train = transaction_only(train)
test = transaction_only(test)
train
                        

,article_id
customer_id,
0,285 3816 4473 8129 16246 18521 18521 19001 220...
1,5472 9970 20312 21672 21686 21706 21715 21715 ...
2,1916 4600 5675 5675 5845 8828 9916 10087 11104...
3,389 2976 3008 3027 3027 3298 3385 5310 5310 56...
4,4056 15902 15904 22047 23302 28498 29450 30122...
...,...
343161,324 326 328 1081 5427 8944 9306 12599 32108 32...
343162,4341 4402 18026 23926 23926 26382 28659 32971 ...
343163,220 300 300 7302 7303 8560 10080 12743 12745 1...


In [44]:
test

,article_id
customer_id,
0,12501 15581 18521 23362 23365 34211
1,21672 21693
2,3075 14561 18922 23981 24555 25417 30895 31841
3,5696 6211 7912 11889 12743 12848 20348 20349 2...
4,15307 30594
...,...
343161,1476 4805 12728 31556 32108 33773
343162,4406 20744 28659
343163,6275 10853 12745 12745 12745


In [46]:
f = open('data/train.txt', 'w')
for customer, articles in enumerate(train.values):
    f.write(f'{customer} {articles[0]}\n')

In [47]:
f = open('data/test.txt', 'w')
for customer, articles in enumerate(test.values):
    f.write(f'{customer} {articles[0]}\n')